![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Praktyczne zastosowania chmury obliczeniowej </h1>
<h2> 3.2. <i>Infrastructure as a Service - zasoby</i>  [laboratoria]</h2> 
<h3> Patryk Żywica (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

# Anatomia zasobów w modelu IaaS

Do tej pory w materiałach na temat modelu IaaS uwaga poświęcona była przede wszystkim maszynom wirtualnym. Oczywiście stanowią one główny zasób obliczeniowy, jednak ich możliwości byłyby ograniczone, gdyby nie usługi dodatkowe skojarzone z nimi.

> **Uwaga**: Nie wszyscy dostawcy IaaS oferują ten sam zakres usług towarzyszących. Do najczęściej oferowanych należą zasoby dyskowe, kopie zapasowe oraz migawki.

Niniejsza część materiałów zostanie poświęcona szczegółowemu omówieniu najczęściej spotykanych usług towarzyszących tj. 
- zasobów dyskowych
- sieci prywatnych i *pływających* adresów IP
- mechanizm równoważenia obciążenia
- kopiom zapasowym i migawkom
- zaporom sieciowym

## Maszyny wirtualne

Jak już wspomniano w poprzedniej części materiałów, maszyny wirtualne są kluczowym elementem IaaS. Zapewniają zasoby obliczeniowe, umożliwiając w ten sposób przetwarzanie danych użytkownika i tych pochodzących z systemów zewnętrznych. 

### Opis 

Pojęcie wirtualizacji jest bardzo szerokie, pokrywające wiele technik budowy maszyn wirtualnych. Pierwsze próby wykorzystania wirtualizacji miały na celu zwiększenie wykorzystania procesora. W latach 60-tych XX wieku firma IBM korzystając z tych techniki pozwoliła na uruchomienie na serwerze typu *mainframe* wielu kopii tego samego systemu operacyjnego co w efekcie pozwalało na wykorzystanie tej samej fizycznej maszyny przez wielu użytkowników jednocześnie [IBM].

![Diagram przedstawia wizualizację pojedynczej maszyny wirtualnej chmurze typu IaaS.](obrazy/zasoby-vm.png)

Możemy w wyróżnić następujące typy wirtualizacji:
- **Pełna wirtualizacja**: nadzorca (oprogramowanie wirtualizacyjne, ang. hypervisor) dostarcza pełne środowisko wirtualne, w którym wirtualizowany system operacyjny (gość, ang. Guest) ma wrażenie, że działa na prawdziwym, fizycznym sprzęcie. W rzeczywistości wszystkie wywołania są przechwytywane przez oprogramowanie i emulowane z wykorzystaniem dostępnego sprzętu. Rozwiązanie te jest najmniej efektywnym typem wirtualizacji ze względu na duży narzut obliczeniowy. Jego główna zaleta to możliwość emulowania środowiska innego niż fizycznie dostępne (np. urządzenia mobilnego na komputerze PC).

- **Wspierana sprzętowo pełna wirtualizacja**: w celu zmniejszenia narzutu obliczeniowego na wirtualizację, część kosztownych operacji dzięki wsparciu sprzętowemu wykonywana jest bezpośrednio na sprzęcie fizycznym. Przykładami takich technologii są Intel VT-x i AMD-V, które są obecnie obsługiwane przez wszystkie większe systemy do wirtualizacji. Kosztem tego rozwiązania jest nieznaczne zmniejszenie bezpieczeństwa oraz utrudniona wirtualizacja innego środowiska niż dostępne fizycznie.

- **Parawirtualiacja**: w tym przypadku system operacyjny gościa musi zostać zaadaptowany do pracy w środowisku wirtualnym, jest zatem świadomy faktu, że pracuje w środowisku wirtualnym. Dzięki temu możliwe jest bardziej efektywne wykorzystanie zasobów i znacznie zmniejszenie narzutu obliczeniowego na wirtualizację. Do przykładów technologii wspierających parawirtualizację należy [VirtIO](https://www.linux-kvm.org/page/Virtio) umożliwiające efektywny dostęp rzeczywistych urządzeń blokowych (dyski), kart sieciowych, a nawet kart graficznych. Wszystkie współczesne systemy operacyjne oparte na jądrze Linux wspierają natywnie VirtIO.

- **Wirtualizacja na poziomie systemu operacyjnego**: często nazywana też konteneryzacją. Pozwala na uruchomienie wielu zwirtualizowanych instancji na jednym komputerze, jednak w odróżnieniu od poprzednich rozwiązań, system operacyjny (a tak właściwie jego jądro) jest współdzielony. Do zapewnienia separacji wykorzystywane są odpowiednie mechanizmy jądra systemu operacyjnego np. `chroot`, `cgroups`, `kernel namespaces`.

W rzeczywistych systemach wirtualizacji techniki te są często łączone. Przy zasobach typu IaaS zawsze oferowana jest pełna wirtualizacja, najczęściej wspierana sprzętowo przez procesor oraz korzystająca z mechanizmu parawirtualizacji dla dysków i sieci. Z punktu widzenia użytkownika zasobów IaaS, nie jest bardzo istotne jak zorganizowane są udostępniane zasoby.

> **Wskazówka**: Jak odróżnić pełną maszynę wirtualną od środowiska oferowanego w ramach wirtualizacji na poziomie systemu operacyjnego? Najpewniejszym sposobem będzie próba zmiany konfiguracji jądra systemu operacyjnego. W przypadku konteneryzacji nie powinno być to możliwe, gdyż jądro jest współdzielone pomiędzy wszystkie instancje.

### Kluczowe cechy

Maszyny wirtualne charakteryzują przede wszystkim wirtualne rdzenie (vCPU), pamięć operacyjna RAM, wirtualna sieć i urządzenia IO. Ponadto dyski często sprzedawane są jako osobna, niezależna od maszyn wirtualnych usługa. Wirtualne procesory (vCPU) mogą nie mieć pokrycia w rzeczywistych rdzeniach procesora. Jest to częsta praktyka (ang. overprovinsioning). Wielu dostawców za odpowiednią dopłatą jest wstanie zagwarantować pełne pokrycie rdzeni (tzw. dedicated cores/vCPU). Niemalże wszystkie zasoby chmurowe mogą być w pewnym stopniu alokowane bez pokrycia. Wyjątek stanowi pamięć operacyjna, która jest głównym ogranicznikiem liczby maszyn wirtualnych per serwer. Obecnie jako standardową platformę usług IaaS przyjmuje się procesor typu x64, pamięć RAM z ECC oraz dyski SSD w konfiguracji RAID. Jednakże, choć nadal niszowe, zasoby obliczeniowe oparte na architekturze ARM zyskują coraz większą popularność, głównie ze względu na ich niski koszt.

Niektórzy dostawcy mają w swojej ofercie specjalistyczne maszyny wirtualne wyposażone w karty graficzne czy specjalistyczne układy obliczeniowe. Są one najczęściej wykorzystywane w przetwarzaniu graficznym oraz w uczeniu maszynowym.


### Sposób wykorzystania

Maszyny wirtualne są uniwersalne, gdyż można na nich instalować dowolne oprogramowanie. Do najczęstszych zastosowań tych zasobów należy zaliczyć:
- serwery webowe (Apache, Nginx) i aplikacyjne (JBoss, Tomcat, Node.js)
- bazy danych (PostgreSQL, MySql)
- obliczenia rozproszone 
- bramy sieciowe, serwery proxy, firewalle

### Dostępne usługi i ich koszt

Poniższa tabela przedstawia miesięczny koszt używania maszyny wirtualnej oferującej 2 vCPU oraz 2GB RAM u różnych dostawców. Jest to jeden z najczęściej alokowanych typów zasobów chmurowych. W przypadku gdy dany dostawca ma w ofercie wiele takich maszyn, przyjęto tę z najpopularniejszej linii produktowej, naliczanie *on-demand* oraz współdzielone zasoby.

W kalkulacjach przyjęto następujące kursy walut:
- 1 EUR = 4.5 PLN
- 1 USD = 3.8 PLN

| Dostawca | Nazwa | Koszt | Uwagi |
| --- | --- | --- | ---- |
| Alibaba | ecs.t6-c1m1.large | 45.83 | 4GB RAM + 20GB SSD|
| AWS | t3.small | 56.90 |  |
| Azure | B2S | 115.20 | 4GB RAM + 8GB SSD |
| Google Cloud Engine | e2-highcpu-2 | 137.21 |
| Hetzner | CPX11 | 18.00 | +40GB SSD |
| Linode | | 76.00 | 4GB RAM + 80GB SSD |
| OVH | d2-4 | 47.53 | 4GB RAM + 50GB SSD |


## Zasoby dyskowe

W bardzo wielu zastosowaniach niezbędne jest przechowywanie danych. W zależności od potrzeb dostępne są różne rodzaje zasobów dyskowych. 

### Opis

Jak już zostało wspomniane zasoby dyskowe bardzo często sprzedawane są jako osobna usługa. Dzięki temu dyski mogą być tworzone i zarządzane niezależnie od maszyn wirtualnych. Przyłączenie dysku do maszyny powoduje, że jest on dostępny dla niej na wyłączność (dysk może być podłączony naraz tylko do jednej maszyny). Możliwe jest jednak odłączenie dysku i podłączenie go do innej maszyny (wielu dostawców umożliwia przełączanie dysków bez potrzeby restartu maszyny).

![Diagram przedstawia wizualizację pojedynczej maszyny wirtualnej chmurze typu IaaS z dołączonym zasobem dyskowym.](obrazy/zasoby-dysk.png)

### Kluczowe cechy

Zasoby dyskowe można scharakteryzować następującymi parametrami:
- **pojemność**: najważniejszy parametr opisujący ilość danych, które można zapisać (mierzona w GB lub TB).
- **prędkość zapisu/odczytu**: informuje o ilości danych jakie będzie można odczytać/zapisać z/na dysku w jednostce czasu (mierzona w MB/s).
- **ilość operacji na sekundę (IOPS)**: informuje o ilości niezależnych operacji zapisu/odczytu jakie można wykonać w jednostce czasu. W ten sposób można określić jaki będzie czas odpowiedzi zasobu na żądanie i ile żądań będzie można obsłużyć jednocześnie (mierzona w operacjach na sekundę, IOPS).
- **stopień redundancji**: informuje w ilu kopiach dane są przechowywane i w jaki sposób są one rozmieszczone fizycznie (ten sam serwer, czy inny, ta sama serwerownia czy nie).

Ważne jest zrozumienie różnic pomiędzy poszczególnymi typami zasobów dyskowych:

- **dyski lokalne w konfiguracji RAID** (ang. local storage): w ofercie znajdują się zarówno dyski HDD (coraz rzadziej) jak i SSD. Takie rozwiązania oznacza, że zasób dyskowy przyłączony do naszej maszyny wirtualnej znajduje się fizycznie w tym samym serwerze co nasza maszyna wirtualna. Takie dyski najczęściej udostępniane są za pośrednictwem parawirtualizacji (VirtIO) i oferują wysoką wydajność zarówno pod względem prędkości zapisu/odczytu (GB/s) jak i ilości operacji na sekundę (IOPS). Przydatne szczególnie dla zastosowań, w których potrzeba niskiego opóźnienia dostępu do dysku np. bazach danych.
- **dyski sieciowe** (ang. network storage): tutaj nadal popularne są dyski twarde (HDD). Dostęp do zasobu odbywa się poprzez sieć dostawcy, co znacznie ogranicza skorzystanie z benefitów szybkich dysków SSD. O ile prędkość zapisu/odczytu nie musi być drastycznie niższa od dysków lokalnych, o tyle ilość operacji na sekundę (IOPS) może być nawet o kilka rzędów wielkości niższa (choć wcale nie musi). W zamian za to dyski sieciowe oferują znacznie większą redundancję danych, dane mogą być przechowywane w wielu kopiach nawet w różnych lokalizacjach geograficznych. Przydatne szczególnie do przechowywania danych wymagających wysokiej dostępności. Dyski takie mogą być przyłączane do maszyny wirtualnej jako udziały sieciowe (np. CIFS) lub jako dyski lokalne (np. iSCSI). Rozwiązanie to obecnie dominuje na rynku.


> **Uwaga** Dostępne są też dyski obiektowe (ang. object storage). Nie można na nich zainstalować systemu operacyjnego ani uruchomić bazy danych. Nadają się jednak doskonale do przechowywania dużych ilości plików (obiektów), które rzadko się zmieniają (zmiana nawet części pliku wymaga nadpisania całości). Zostaną one omówione w module poświęconym modelowi SaaS chmury obliczeniowej.

### Sposób wykorzystania

Każda maszyna wirtualna potrzebuje przynajmniej jednego zasobu dyskowego, tego na którym zainstalowany zostanie system operacyjny i wymagane oprogramowanie. Jak zostanie to opisane w dalszej części materiałów, instalacja systemu, oprogramowanie i ich konfiguracja bardzo często jest w pełni zautomatyzowana, a co za tym idzie dysk ten nie jest kluczowym zasobem. W bardzo wielu przypadkach wystarcza pojemność rzędu 8-20GB.

Jeśli maszyna wirtualna potrzebuje zapisywać jakieś dane, które mają być przechowywane długoterminowo rekomendowane jest dołączanie do niej osobnego zasobu dyskowego. W ten sposób będzie można łatwo przenieść dane pomiędzy maszynami. Ułatwia to też zarządzanie kopią zapasową i migawkami.

> **Uwaga**: Należy też zaznaczyć, że niektórzy dostawcy (np. AWS) nie oferują żadnych zasobów dyskowych w standardzie. W takim przypadku warto zaalokować 2 osobne zasoby, tak aby jeden zawierał system i oprogramowanie (czyli część specyficzną dla danej maszyny, która nie będzie przełączana) oraz a drugi zawierał przechowywane dane (które można będzie łatwo przenieść pomiędzy maszynami).

Większość dostawców pozwala, dowolnie zmieniać rozmiar zasobów dyskowych (często bez konieczności restartu maszyny wirtualnej). Mimo wszystko zmiana rozmiaru dysku nie jest łatwa i automatyczna. Trudność ta wynika z konieczności zmiany rozmiaru systemu pliku. Samo zmniejszenie/zwiększenie dostępnej przestrzeni dyskowej nie wystarczy, aby system operacyjny zauważył tę zmianę. Konieczne jest ręczne dopasowanie systemu plików do nowych warunków. Jest to proces potencjalnie niebezpieczny i czasochłonny, który może się wiązać z przerwą w dostępności do systemu, a nawet utratą danych (szczególnie w przypadku zmniejszania systemu plików).

### Dostępne usługi i ich koszt

Zasoby dyskowe nie są usługą, która można wykorzystać poza maszyną wirtualną u danego dostawcy. Stąd wybierając dostawcę należy jednocześnie uwzględnić oferowane zasoby dyskowe i ich parametry (w tym cenę). W poniższej tabeli zebrano informacje o zasobach dyskowych oferowanych przez dostawców. Koszt podany jest za miesiąc w przeliczeniu na 1GB.

| Dostawca | Typ | Koszt | Uwagi |
| --- | --- | --- | --- |
| Alibaba | oba, SSD | 0.22 | każda maszyna wirtualna posiada dysk lokalny w standardzie |
| Azure | sieciowe, SSD | 0.29 | każda maszyna wirtualna posiada dysk lokalny w standardzie |
| AWS | sieciowe, SSD | 0.38 | dyski lokalne dostępne tylko dla wybranych typów maszyn wirtualnych |
| Google Cloud | sieciowe, HDD | 0.15 | |
| Google Cloud | sieciowe, SSD | 0.64 | |
| Google Cloud | lokalne, SSD | 0.30 | zasoby dostępne tylko w jednostkach po 375GB |
| Hetzner | sieciowe, SSD | 0.18 | każda maszyna wirtualna posiada dysk lokalny w standardzie |
| Linode | sieciowe, SSD | 0.38 | każda maszyna wirtualna posiada dysk lokalny w standardzie |
| OVH | sieciowe, SSD | 0.17 | każda maszyna wirtualna posiada dysk lokalny w standardzie |

## Wirtualne sieci prywatne

Budując systemy informatyczne, często wyróżniamy w nich niezależne komponenty (np. główna aplikacja i baza danych). Wdrażając aplikację w modelu IaaS warto każdy z komponentów umieścić na osobnej maszynie wirtualnej. Konieczny jest zatem mechanizm komunikacji pomiędzy komponentami. Standardowo każda maszyna wirtualna dysponuje publicznym adresem IP, który oczywiście można wykorzystać do takiej komunikacji. Niestety rozwiązanie takie wymaga odpowiedniej konfiguracji i zabezpieczenia udostępnianych usług. Udostępnienie publicznego dostępu do np. bazy danych systemu, nigdy nie jest dobrym rozwiązaniem. Odpowiedzią na ten problem są wirtualne sieci prywatne.

### Opis

Wirtualna sieć prywatna dysponuje własną pulą adresów IP, najczęściej jako podzbiór zakresu `10.0.0.0/8` (np. `10.10.10.0/24`). Maszyny wirtualne mogą zostać dodane do tej sieci, dzięki czemu otrzymają adresy prywatne z jej puli, które następnie mogą wykorzystać do komunikacji wewnętrznej. Technicznie taka sieć jest bytem w pełni wirtualnym, nie istnieje fizycznie. Dane przekazywane są tymi samymi kanałami i z wykorzystaniem tego samego sprzętu co w przypadku komunikacji publicznej pomiędzy tymi samymi maszynami wirtualnymi. Jednak dodanie warstwy logicznej (wirtualnej) separacji zasobów znacznie upraszcza jej strukturę. Ponieważ sieć prywatna nie jest dostępna na zewnątrz ani dla innych klientów dostawcy usług chmurowych, można znacznie uprościć konfigurację i zabezpieczenie komponentów zlokalizowanych w tej sieci.

> **Wskazówka**: Zapis `10.10.10.0/24` to tak zwana notacja CIDR. Pozwala ona dokładnie zdefiniować pulę adresów dostępnych w sieci. Pierwsza część `10.10.10.0` określa bazowy adres IP. Druga część `24` określa maskę podsieci, informując o tym, ile pierwszych bitów adresu IP wskazuje adres podsieci. Pozostałe `32-24=8` bity używane są do adresacji hostów w sieci (w tym przypadku dostępne jest 256 adresów IP). Zapis `10.10.10.0/24` oznacza, że adres podsieci to 10.10.10.0, a maska podsieci to 255.255.255.0 co w praktyce oznacza pulę adresów `10.10.10.0 - 10.10.10.255`.

> **Uwaga**: Termin wirtualna sieć prywatna (ang. virtual private network, VPN) może być mylący, ze względu na wiele alternatywnych znaczeń. Termin VPN najczęściej wykorzystywany jest w kontekście tunelowania prywatnego ruchu sieciowego (np. dostępu do wewnętrznych zasobów przedsiębiorstwa) przez publiczną sieć (np. Internet). Ostatnio popularne stało się też użycie terminu VPN w kontekście anonimizacji komunikacji sieciowej.

![Diagram przedstawia logiczny podział sieci komputerowej u dostawcy usługi IaaS. Zaznaczono pulę publicznych adresów IP, do których przyłączone są maszyny wirtualne. Ponadto zaznaczono wirtualne sieci prywatne, w ramach, których obowiązuje oddzielna adresacja IP. Maszyny wirtualne mogą być przypisane do dowolnej kombinacji publicznych i prywatnych adresów IP.](obrazy/zasoby-siec.png)

Z koncepcją prywatnej sieci wirtualnej łączy się też pojęcie *pływających adresów IP* (ang. floating IP). Tworząc nową maszynę wirtualną z dostępem do sieci publicznej (co jest najczęściej opcją domyślną), każdorazowo przydzielany jest jej losowy adres IP z dostępnej puli. W niektórych zastosowaniach częste zmienianie adresu IP nie jest dopuszczalne. Przykładowo serwer WWW powinien mieć stały adres IP ze względu na duże opóźnienia w propagacji wpisów DNS. W takiej sytuacji można skorzystać z *pływającego adresu IP*, który może być przypisany do dowolnej maszyny wirtualnej. Takie przypisanie można też dowolnie modyfikować, z niemalże natychmiastowym efektem. Pozwala to zachować stały adres IP, nie ograniczając jednocześnie możliwości zmiany i tworzenia nowych maszyn wirtualnych.

### Sposób wykorzystania

Tworząc system informatyczny, który ma zostać wdrożony w modelu IaaS należy zaprojektować go tak aby kluczowe komponenty mogły znajdować się na osobnych maszynach wirtualnych. Następnie należy dokonać podziału tych komponentów/maszyn wirtualnych na takie, do których niezbędny jest dostęp z zewnątrz (dostarczają usługi dla systemów zewnętrznych i użytkowników) oraz takie, które takiego dostępu nie wymagają. Wszystkie maszyny wirtualne w ramach danego systemu umieszczamy w jednej wirtualnej sieci prywatnej i tą sieć wykorzystujemy do komunikacji wewnętrznej. Tylko maszyny wirtualne z pierwszej grupy otrzymują publiczne adresy IP. W ten sposób można łączyć prywatną komunikację (np. z bazą danych) z publiczną komunikacją w Internecie (np. z użytkownikami). Ponadto dla każdej maszyny wirtualnej istnieje możliwość określania jakie oprogramowanie nasłuchuje na danym adresie IP, co umożliwia oferowanie publicznych i prywatnych usług w ramach pojedynczej maszyny. Ponadto, jeśli stabilność adresów IP jest istotna, wszystkie publiczne adresy IP powinny być skonfigurowane jako *pływające*.

### Dostępne usługi i ich koszt

U wszystkich znaczących dostawców usługa wirtualnej sieci prywatnej jest darmowa. Ponadto, całość ruchu w tej sieci traktowana jest jako wewnętrzna i nie są za nią naliczane opłaty. *Pływające adresy IP* związane są z kosztem naliczanym najczęściej za każdą godzinę rezerwacji adresu nie zależnie od tego czy taki adres jest w użyciu czy też nie. Koszt tej usługi należy szacować na poziomie 10-15 zł miesięcznie.

Nie ma jednolitej nazwy na ten typ usług. Poniżej podane zostały nazwy usługi sieci wirtualnej i *pływających adresów IP* u różnych dostawców:

| Dostawca | Wirtualna sieć prywatna | Pływający adres IP |
| --- | --- | --- |
| Alibaba | Virtual Private Cloud (VPC) | Elastic IP |
| AWS | Virtual Private Cloud (VPC) | Elastic IP |
| Azure | Azure Virtual Network (VNet) | Statyczny adres IP |
| Google Cloud | Virtual Private Cloud (VPC) | Static IP |
| Hetzner | Networks | Floating IP |
| Linode | Private VLAN | niedostępne |
| OVH | Private Network | Floating IP |

## Zapory sieciowe

Każda maszyna wirtualna może mieć zainstalowaną i skonfigurowaną własną zaporę sieciową. Takie rozwiązanie może jednak prowadzić do problemów w zarządzaniu wieloma serwerami jednocześnie. Stąd większość dostawców oferuje możliwość zdefiniowania zapory sieciowej, która umożliwia filtrowanie ruchu sieciowego jeszcze zanim trafi on do maszyny wirtualnej.

### Kluczowe funkcje

Zapora sieciowa zasadniczo definiuje reguły określające jaki ruch sieciowy jest dozwolony, a jaki nie. Każda reguła może określać:
- typ adresu: IPv4 lub IPv6
- protokół: TCP, UDP, ICMP i inne
- źródłowy adres IP
- docelowy port (lub zakres) w przypadku protokołów TCP i UDP

U niektórych dostawców możliwe jest filtrowanie zarówno ruchu przychodzącego jak i wychodzącego.

### Sposób użycia

Dostawca usługi IaaS pozwala zdefiniować zbiory reguł zapory sieciowej. Taki zbiór może być następnie przypisany do wybranych maszyn wirtualnych. Dzięki temu możliwe jest jednoczesne zarządzanie zaporą sieciową wielu maszyn jednocześnie jak i współdzielenie ich konfiguracji. Każda maszyna wirtualna z przypisanym publicznym adresem IP powinna zostać objęta zaporą sieciową oferowaną przez dostawcę usługi, ponadto w takim przypadku nie ma już potrzeby definiowania zapory na poziomie systemu operacyjnego, chyba, że wymagane jest osiągnięcie niestandardowego zachowania (np. pukanie do portów, ang. port knocking).

Najczęściej domyślnie cały ruch wychodzący jest dozwolony, podczas gdy ruch przychodzący jest zablokowany. Aby umożliwić dostęp z zewnątrz do serwisów oferowanych na danej maszynie wirtualnej należy dodać odpowiednie reguły.

### Dostępne usługi i ich koszt

Usługa zapory sieciowej u większości dostawców jest darmowa. Różne jest jednak nazewnictwo. Należy też zwrócić uwagę, że często w ofercie znajdują się osobne produkty o tej nazwie, które służą jednak innym celom niż te opisane powyżej (są to zapory sieciowe na poziomie sieci a nie pojedynczych maszyn wirtualnych). Poniższa tabela przedstawia nazwy u poszczególnych dostawców:

| Dostawca | Zapora sieciowa |
| --- | --- |
| Alibaba | Security group |
| AWS | Security group |
| Azure | Security group |
| Google Cloud |  firewall rules |
| Hetzner | Firewall |
| Linode | Cloud Firewall |
| OVH | Network Firewall |

# Równoważenie obciążenia

We wcześniejszej części materiałów omówiony został mechanizm skalowania horyzontalnego, polegającego na zwiększeniu liczby niezależnych maszyn wirtualnych obsługujących żądania. Z drugiej strony użytkownicy systemu z pewnością potrzebują pojedynczego punktu dostępowego. Taki punkt dostępowy najczęściej ma postać adresu URL, domenowego lub IP systemu. Najprostszy mechanizm skalowania horyzontalnego można zrealizować na poziomie wpisów DNS. Protokół DNS umożliwia wskazanie wielu adresów IP dla danej domeny wraz z priorytetami. Takie rozwiązanie jest jednak mało elastyczne ze względu  na powolną propagację wpisów DNS. Stąd istnieje potrzeba zapewniania bardziej dynamicznego mechanizmu skalowania.

### Opis

**Mechanizm równoważenia obciążenia** (ang. load balancer) to najczęściej specjalistyczne oprogramowanie dostarczane przez usługodawcę chmury, które jest zoptymalizowane pod kątem przyjmowania dużego ruchu sieciowego i przekazywania go do innych maszyn wirtualnych (węzłów, ang. nodes). Takie rozwiązanie może działać samodzielnie, lub stanowić mechanizm drugiego poziomu razem z równoważeniem opartym na wpisach DNS.

![Diagram przedstawia wirtualna sieć prywatną w której wszystkie maszyny wirtualne posiadają wyłącznie prywatny adres IP. Komunikacja z Internetem odbywa się za pośrednictwem mechanizmu równoważenia obciążenia. Ponadto ruch sieciowy jest filtrowany za pośrednictwem zapory sieciowej.](obrazy/zasoby-lb.png)

### Kluczowe funkcje i cechy

Mechanizm równoważenia obciążenia to wbrew pozorom dość złożone oprogramowanie. Kluczowe jego funkcje i cechy to:

1. **Obsługiwane protokoły**: W rozwiązaniach praktycznych najczęściej zrównoleglanie obciążenia odbywa się na następujących warstwach modelu ISO-OSI: 
    - 4 (transmisji) - najbardziej ogólny, pozwala równoważyć dowolne połączenia TCP/UDP. Mechanizm równoważenia obciążenia nie wykorzystuje wiedzy na temat przesyłanych danych.
    - 7 (aplikacji) - najczęściej dotyczy protokołu HTTP, pozwalając na jednokierunkowy, bezstanowy transfer danych. Mechanizm równoważenia obciążenia wykorzystuje wiedzę zawartą w przesyłanych danych, w szczególności może korzystać z nagłówków HTTP, kodów odpowiedzi itp.

    W kontekście równoważenia obciążenia kluczowe jest też rozróżnienie pomiędzy stanowymi i bezstanowymi połączeniami. Komunikacja TCP i UDP wymaga utrzymania połączenia przez cały czas trwania komunikacji, która nie jest ograniczona czasowo. W ten sposób serwer zrównoleglający przez cały czas utrzymuje aktywne połączenie co znacznie ogranicza liczbę jednoczesnych połączeń i zwiększa zużycie zasobów.

    W przypadku komunikacji HTTP, choć technicznie wykorzystuje ona również protokół TCP, to połączenie jest zamykane gdy tylko cała odpowiedź na żądanie HTTP zostanie przesłana. Dalsza komunikacja wymaga ponownego nawiązania połączenia. W ten sposób można znacznie bardziej efektywnie utrzymywać bardzo wielu klientów jednocześnie. Pojawia się natomiast inny problem, w przypadku niektórych systemów (np. korzystających ze stanowej sesji użytkownika) konieczne może się okazać, aby kolejne żądania od tego samego klienta obsługiwane były przez ten sam węzeł.

1. **Dostępne strategie zrównoleglania**: Mechanizm zrównoleglania obciążenia dokonuje wyboru, który węzeł obsłuży bieżące żądanie. Wyróżnić można kilka najpopularniejszych strategii:
    - losowy - węzeł wybierany jest losowo
    - round robin - węzły ustawione są w listę, która przechodzona element po elemencie, a w momencie dotarcia na koniec algorytm ponownie przechodzi na początek listy
    - najmniejsza liczba aktywnych połączeń - szczególnie przydatny przy komunikacji TCP
    - obciążenie węzła - w oparciu o wybrane parametry serwera (np. użycie CPU) wybierany jest serwer najmniej obciążony
    - sesyjne - wybierany jest węzeł, który obsługiwał poprzednie żądania z tego samego źródła

1. **Sposoby weryfikacji sprawności węzłów**: Do poprawnego działania, konieczna jest metoda określenia czy dany węzeł działa poprawnie. Wyróżnić można dwie strategie:
    - prewencyjne - mechanizm zrównoleglania obciążenia w ustalonym odstępie czasu dokonuje sprawdzenia stanu węzła (np. poprzez wysłanie odpowiedniego żądania HTTP)
    - reaktywne - monitorowane są odpowiedzi zwracane do klientów, w przypadku wystąpienia wielu podejrzanych odpowiedzi (np. kod HTTP 5XX), węzeł uznawany jest za nie działający

1. **Przepustowość, redundancja i liczba jednocześnie obsługiwanych połączeń**: To jest parametr, który należy dostosować do potrzeb systemu. Niezależnie od wydajności wszystkich węzłów, system nie obsłuży więcej żądań niż jest w stanie to zrobić mechanizm zrównoleglający. Ponadto tworzy to w systemie tak zwany pojedynczy punkt awarii (ang. single point of failure). Awaria mechanizmu zrównoleglającego uniemożliwia korzystanie z systemu.

1. **TLS offloading**: Coraz częściej mechanizm zrównoleglania oferuje dodatkową funkcję jaką jest dodanie szyfrowania TLS do komunikacji. Najczęściej jest to wykorzystywane w przypadku, gdy węzły wykorzystują do komunikacji protokół HTTP, natomiast klient oczekuje komunikacji zaszyfrowanej (HTTPS). Mechanizm zrównoleglania obciążenia może w takiej sytuacji tłumaczyć w locie pomiędzy HTTP i HTTPS.


### Sposób użycia
Korzystając z wirtualnych sieci prywatnych, nie należy przypisywać publicznych adresów do żadnych maszyn wirtualnych. Jedyne punkty dostępowe do systemu powinny być udostępnione za pośrednictwem mechanizmu równoważenia obciążenia i to adres tej usługi powinien być dostępny publicznie. W ten sposób oprócz zwiększonego bezpieczeństwa, otrzymujemy pełną dowolność w budowaniu naszej infrastruktury.

Ponadto w przypadku dużych systemów, systemów rozproszonych geograficznie albo o wymaganej bardzo dużej dostępności, warto rozważyć połączenie mechanizmu równoważenia obciążenia z wykorzystaniem technik opartych o wielokrotne wpisy DNS wskazując na zasoby dostępne w odseparowanych infrastrukturach.

### Dostępne usługi i ich koszt
Praktycznie każdy dostawca IaaS oferujący usługę sieci prywatnej oferuje też mechanizm równoważenia obciążenia. Jest to najczęściej usługa płatna o koszcie zbliżonym do 1-2 rdzeniowej współdzielonej maszyny wirtualnej. Ponadto niektóre usługi opierają swój koszt na bazie zużytych zasobów (tzw. LCU, ang. Load Balancer Capacity Units). LCU obejmuję pewną ilość zasobów przydzielonych do mechanizmu równoważenia obciążenia, najczęściej:
- pewną liczbę nowych połączeń w każdej sekundzie (np. 25)
- pewną liczbę aktywnych połączeń na minutę (np. 3000)
- pewnę ilość transferu (najczęściej 1GB)


| Dostawca | Nazwa | Warstwa | Koszt miesięczny |
| --- | --- | --- | --- |
| Alibaba | Classic Load Balancer | 4 | 8.21 + 16.42 za transfer do Mbit/s |
| Alibaba | Application Load Balancer | 7 | 19.15 + 19.15 za każde LCU |
| AWS | Classic Load Balancer | 4 | 68.40 + 0.03 za GB |
| AWS | Application Load Balancer | 7 | 61.56 + 21.89 za każde CLU |
| Azure | Load balancer | 4 | 68.40 + 0.02 za GB |
| Azure | Application Gateway | 7 | 69.35 +  + 21.89 za każde CLU|
| Google Cloud | Cloud Load Balancing | 4 i 7 | 68.40 + 0.03 za GB |
| Hetzner | Load balancer | 4 i 7 | 26.24 |
| Linode | NodeBalancer | 4 i 7 | 38.00 |
| OVH | Network Firewall | 7 | 88.99 |

## Kopie zapasowe

Większość usług chmurowych oferuje bardzo wysoką dostępność i redundancję. Nie usprawiedliwia to jednak braku kopi zapasowej kluczowych danych. W przypadku usług chmurowych podobnie jak w przypadku klasycznych środowisk wirtualnych mamy do czynienia z dwoma głównymi technikami zabezpieczania danych z maszyny wirtualnej: klasycznej kopi zapasowej na zewnętrznym serwerze oraz migawek całej maszyny wirtualnej.

![Diagram przedstawia maszynę wirtualną wraz z migawkami przyłączonego do niej zasobu dyskowego.](obrazy/zasoby-snapshot.png)

### Opis

**Klasyczna kopia zapasowa** polega na cyklicznym przesyłaniu kluczowych danych z maszyn wirtualnych na zewnętrzne serwery. Taka kopia zapasowa wymaga ręcznej konfiguracji, może natomiast obejmować bardziej specyficzne dane takie jak, wyeksportowaną kopię bazy danych w odpowiednim formacie. Dla uproszczenia można przyjąć, że taka kopia to pojedyncze spakowane archiwum. Odtwarzanie z takiej kopi również wymaga odpowiedniej konfiguracji i wykorzystania odpowiednich narzędzi. Klasyczna kopia zapasowa powinna być przechowywana w infrastrukturze możliwie bardzo odseparowanej od głównej. Warto rozważyć inną lokalizację geograficzną a nawet dostawcę usług.

**Migawka maszyny wirtualnej** to z drugiej strony kopia wszystkich zasobów dyskowych maszyny wirtualnej czasami powiązana również z jej konfiguracją (typem instancji, dostępnymi zasobami obliczeniowymi, itp.). Migawka obejmuje również pliki systemu operacyjnego oraz zainstalowane aplikacje. Bazując na migawce można bardzo łatwo odtworzyć maszynę wirtualną lub stworzyć jej identyczną kopię. Technicznie, z powodu dużego rozmiaru tego typu migawki przechowywane są najczęściej w sposób przyrostowy (każda kolejna migawka zachowuje tylko różnice względem poprzedniej migawki).

Można odnieść wrażenie, że migawki systemu są rozwiązaniem znacznie bardziej elastycznym. Do głównych wad należy ich kosztowność w przypadku długoterminowego przechowywania danych. Rozmiar migawek ma tendencje do szybkiego wzrostu. Nawet zastosowanie migawek przyrostowych nie rozwiązuje w pełni tego problemu. Druga wada dotyczy spójności danych. Migawka systemu utrwala jego bieżący stan, który niekoniecznie musi być spójny. Dlatego zalecane jest robienie migawek, gdy maszyna wirtualna jest w stanie wyłączonym. To z kolei powoduje przestoje w działaniu systemu i ogranicza zastosowanie jako efektywny sposób robienia kopi zapasowej.


### Sposób użycia

Zaleca się korzystanie z obu strategii kopii zapasowej. Migawki maszyny wirtualnej należy wykorzystywać do krótkotrwałego zabezpieczania danych i całego stanu systemu, głównie ze względu na to, że zapewniają szybki i prosty sposób przywracania po awarii. Klasyczne kopie zapasowe należy stosować do długoterminowego przechowywania danych oraz zapewniania bardzo wysokiej dostępności na wypadek awarii. Kluczowe jest zapewnienie separacji fizycznej kopii zapasowej od głównej infrastruktury. Ponadto należy się zabezpieczyć przed sytuacją, w której wadliwa/zaatakowana maszyna wirtualna zyskuje dostęp do serwera kopii zapasowej. Szczególnie niekorzystne są dwa scenariusze: usunięcie kopii zapasowej oraz uzyskanie dostępu do kopii zapasowej innych usług/systemów.


### Dostępne usługi i ich koszt

Praktycznie wszyscy dostawcy usług chmurowych oferują oba rodzaje kopii zapasowej. Najczęściej są to usługi płatne, z naliczaniem za każdy przechowywany GB danych. Poniższa tabela porównuje miesięczny koszt różnych usług.

| Dostawca | Nazwa migawki | Koszt za GB migawki | Nazwa klasyczna kopia | Koszt GB klasycznej kopii |
| --- | --- | --- | --- | --- |
| Alibaba | Snapshots | 0.76 | Object Storage Service Infrequent Access | 0.068 |
| AWS | Classic Load EBS Snapshots | 0.19 | S3 Infrequent Access | 0.048 |
| Azure | Snapshots | 0.19 | Blob Storage Cold | 0.076 |
| Google Cloud | Snapshot | 0.099 | Cloud Coldline Storage | 0.015 |
| Hetzner | Snapshot | 0.054 | Backups | 20% ceny maszyny wirtualnej |
| Linode | Images | 0.38 | Object Storage | 0.076 |
| OVH | Volume Snapshot | 0.072 | Object Storage |  0.072 |

# Literatura
* [IBM]: R. J. Creasy. The Origin of the VM/370 Time-sharing System. IBM. 1981
* [ALI]: Alibaba. [Cennik usług Alibaba Cloud](https://www.alibabacloud.com/pricing). Dostęp 28.09.2021
* [AWS]: Amazon. [Cennik usług AWS](https://aws.amazon.com/pricing/). Dostęp 28.09.2021
* [AZ]: Microsoft. [Cennik usług Azure](https://azure.microsoft.com/pl-pl/pricing/). Dostęp 28.09.2021
* [GC]: Google. [Cennik usług Google Cloud](https://cloud.google.com/pricing). Dostęp 28.09.2021
* [HZ]: Hetzner. [Cennik usług Hetzner Cloud](https://www.hetzner.com/cloud). Dostęp 28.09.2021
* [LIN]: Linode. [Cennik usług Linode](https://www.linode.com/pricing/). Dostęp 28.09.2021
* [OVH]: OVH. [Cennik usług OVH](https://www.ovhcloud.com/pl/public-cloud/prices/). Dostęp 28.09.2021
